In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Train file: CSV containing the customers for whom loan eligibility is known as 'Loan_Status'

Loan_ID - Unique Loan ID

Gender  - Male/ Female

Married - Applicant married (Y/N)

Dependents - Number of dependents

Education - Applicant Education (Graduate/ Under Graduate)

Self_Employed - Self employed (Y/N)

ApplicantIncome - Applicant income

CoapplicantIncome - Coapplicant income

LoanAmount - Loan amount in thousands

Loan_Amount_Term - Term of loan in months

Credit_History - credit history meets guidelines

Property_Area - Urban/ Semi Urban/ Rural

Loan_Status - (Target) Loan approved (Y/N)

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#### Importing the data

In [ ]:
Loan_train=pd.read_csv('../input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv')

#### Looking at the Dataset

In [ ]:
Loan_train.head()

##### Shape of the Dataset

In [ ]:
print('Number of Rows in the Data',Loan_train.shape[0])
print('Number of Columns in the Data',Loan_train.shape[1])

In [ ]:
Loan_train.info()

* We observe there are Null values in the Data 
* Credit_History and Loan_Amount_Term are Categorical varible we need to change its Data type
* Loan_Status is the Target Variables
* We can drop the Loan_ID Column as it does not provide any information

#### Checking for duplicate values

In [ ]:
dups=Loan_train.duplicated()
print('Number of duplicate records',dups.sum())

In [ ]:
Loan_train['Credit_History']=Loan_train['Credit_History'].astype('object')
Loan_train['Loan_Amount_Term']=Loan_train['Loan_Amount_Term'].astype('object')

In [ ]:
Loan_train['Dependents']=np.where(Loan_train['Dependents']=='0',0,Loan_train['Dependents'])
Loan_train['Dependents']=np.where(Loan_train['Dependents']=='1',1,Loan_train['Dependents'])
Loan_train['Dependents']=np.where(Loan_train['Dependents']=='2',2,Loan_train['Dependents'])
Loan_train['Dependents']=np.where(Loan_train['Dependents']=='3+',3,Loan_train['Dependents'])

In [ ]:
Loan_train['Dependents']=Loan_train['Dependents'].astype('float64')

In [ ]:
Loan_train.drop(['Loan_ID'],axis=1,inplace=True)

#### Separating object and numeric data

In [ ]:
cat=[]
num=[]
for i in Loan_train.columns:
    if Loan_train[i].dtype=='object':
        cat.append(i)
    else:
        num.append(i)

print(cat)
print(num)

In [ ]:
#Looking at unique values
for i in Loan_train[cat]:
    print(i)
    print( Loan_train[i].value_counts())
    print('**********************************')

#### Summary of the Numeric Data

In [ ]:
Loan_train[num].describe().T

*  we observe that there is high standard deviation i.e there is a lot of variance in the Dataset.
*  some Records are missing in LoanAmount and Loan_Amount_Term columns
*  Maximum Loan Amount  applied for is 700 thousand

#### Summary of the Non - Numeric Data

In [ ]:
Loan_train[cat].describe().T

* Most Applicants are Male
* Most applicant are Married
* Most Applicants have no dependents
* Most Applicants are not Self Employed
* Most Applicants are from Semi urban
* MOst of the Loans have been Approved
* Minimum Term of loan in months is 12 and Maximum is 480 months.

#### Checking for NUll Values

In [ ]:
Loan_train[num].isnull().sum()

In [ ]:
Loan_train[cat].isnull().sum()

In [ ]:
#We observe that there are Null values in the Dataset

In [ ]:
#Updating the values in  Tagret Variable

In [ ]:
Loan_train['Loan_Status']=np.where(Loan_train['Loan_Status']=='Y',1,Loan_train['Loan_Status'])
Loan_train['Loan_Status']=np.where(Loan_train['Loan_Status']=='N',0,Loan_train['Loan_Status'])

In [ ]:
Loan_train['Loan_Status']=Loan_train['Loan_Status'].astype('int64')

#### Looking at the distribution of the Dataset

In [ ]:
for i in Loan_train[num]:
    plt.figure(figsize=(8,6))
    sns.histplot(Loan_train[i],kde=True)

* We observe that the numeric Columns are highly Skewed towards right

#### Univariate Analysis

In [ ]:
plt.subplot(1,1,1)
Loan_train['Loan_Status'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%')
plt.show()
plt.subplot(1,2,2)
sns.countplot(Loan_train['Loan_Status'])
plt.show()

In [ ]:
#68.7% of the Loans get approved while 31.3% get rejected

In [ ]:
Loan_train['Gender'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%')
plt.show()
sns.countplot(Loan_train['Gender'])
plt.show()

In [ ]:
#81.4% of the applicants are Male

In [ ]:
Loan_train['Self_Employed'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%')
plt.show()
sns.countplot(Loan_train['Self_Employed'])
plt.show()

In [ ]:
#85.9% of the applicants are Self_Employed

In [ ]:
Loan_train['Dependents'].value_counts().plot.pie(autopct='%1.1f%%')
plt.show()
sns.countplot(Loan_train['Dependents'])
plt.show()

In [ ]:
# 57.6% of the Applicants have no dependents
# Most Applicant have no Dependents

In [ ]:
Loan_train['Education'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%')
plt.show()
sns.countplot(Loan_train['Education'])

In [ ]:
#78.2% of the Applicants are Graduates

In [ ]:
Loan_train['Credit_History'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%')
plt.show()
sns.countplot(Loan_train['Credit_History'])
plt.show()

In [ ]:
#84.2% of the Applicants have very good Credit Score

In [ ]:
Loan_train['Property_Area'].value_counts().plot.pie(autopct='%1.1f%%')
plt.show()
sns.countplot(Loan_train['Property_Area'])
plt.show()

In [ ]:
# Most Applicants are from Urban  or Semiurban Area

# Bivariate Analysis

#### Looking at the aproval rate for gender

In [ ]:
sns.countplot(Loan_train['Gender'],hue=Loan_train['Loan_Status'])
plt.show()

In [ ]:
pd.crosstab(Loan_train['Gender'],Loan_train['Loan_Status'],margins=True)

In [ ]:
print('Approaval rate of Loans of Male Applicants',np.round(339/489,2))
print('Approaval rate of Loans of Female Applicants',np.round(75/112,2))

In [ ]:
#Although Female Applicants are much less, does not mean that their Loans are less likely to be approved.

#### Looking at the aproval rate with Respect to Marital Status 

In [ ]:
sns.countplot(Loan_train['Married'],hue=Loan_train['Loan_Status'])
plt.show()

In [ ]:
pd.crosstab(Loan_train['Married'],Loan_train['Loan_Status'],margins=True)

In [ ]:
print('Approaval rate of Loans of Married Applicants',np.round(285/398,2))
print('Approaval rate of Loans of Non-Married Applicants',np.round(134/213,2))

In [ ]:
#Loans of Married Applicants are more likely to be approved

#### Looking at the aproval rate with Respect to Dependents

In [ ]:
sns.countplot(Loan_train['Dependents'],hue=Loan_train['Loan_Status'])
plt.show()

In [ ]:
pd.crosstab(Loan_train['Dependents'],Loan_train['Loan_Status'],margins=True,normalize=True)

In [ ]:
#Applicants with 0 Dependents have more chances of Loan Approval, while 3+ Dependents have very low chances

#### Looking at the aproval rate with Respect to Education

In [ ]:
sns.countplot(Loan_train['Education'],hue=Loan_train['Loan_Status'])
plt.show()

In [ ]:
pd.crosstab(Loan_train['Education'],Loan_train['Loan_Status'],margins=True)

In [ ]:
print('Approaval rate of Loans of Graduate Applicants',np.round(340/480,2))
print('Approaval rate of Loans of Not Graduate Applicants',np.round(82/134,2))

In [ ]:
#Applicants who are Graduate have more chances of Loan Approval.

#### Looking at the aproval rate with Respect to Credit Score

In [ ]:
sns.countplot(Loan_train['Credit_History'],hue=Loan_train['Loan_Status'])
plt.show()
pd.crosstab(Loan_train['Credit_History'],Loan_train['Loan_Status'],margins=True)

In [ ]:
print('Approaval rate of Loans for Applicants who do not have credit history meets guidelines',np.round(7/89,2))
print('Approaval rate of Loans for Applicants who have credit history meets guidelines',np.round(378/475,2))

In [ ]:
#As we Expect Applicants with good credit History have more chances of Loan Approval while for Applicants with Poor Credit History there is almost no individual.

#### Looking at the aproval rate with Respect to Property Area

In [ ]:
sns.countplot(Loan_train['Property_Area'],hue=Loan_train['Loan_Status'])
plt.show()

In [ ]:
pd.crosstab(Loan_train['Property_Area'],Loan_train['Loan_Status'],margins=True)

In [ ]:
print('Approaval rate of Loans for Applicants from Rural Area',np.round(110/179,2))
print('Approaval rate of Loans for Applicants from Semiurban Area',np.round(179/233,2))
print('Approaval rate of Loans for Applicants from Urban Area',np.round(133/202,2))

In [ ]:
#Applicants from Semiurban Area have more chances of Loan Approval while for Applicants from Rural area have least.

#### Looking at the aproval rate with Respect to Applicant Income.

In [ ]:
sns.kdeplot('ApplicantIncome',hue='Loan_Status',data=Loan_train)

In [ ]:
sns.barplot('Loan_Status','ApplicantIncome',data=Loan_train)

In [ ]:
# We Would Expect Applicant with Higer Income have more chances for their Loans to be approved
# But as we can see Applicants with Higher Income also have thier Loans Rejected.
# NOthing can be concluded on the relation b/w Loan_Status and ApplicantIncome

#### Looking at the aproval rate with Respect to Co-applicant Income.

In [ ]:
sns.kdeplot('CoapplicantIncome',hue='Loan_Status',data=Loan_train)

In [ ]:
sns.barplot('Loan_Status','CoapplicantIncome',data=Loan_train)
plt.show()

In [ ]:
# We Would Expect Applicant with Co applicant Income Income have more chances for their Loans to be approved as they are less likely to Default
# Nothing can be concluded on the relation b/w Loan_Status and Co applicant Income

#### Looking at the aproval rate with Respect to  Loan Amount

In [ ]:
sns.kdeplot('LoanAmount',hue='Loan_Status',data=Loan_train)

In [ ]:
# Nothing can be concluded on the relation b/w Loan_Status and Co applicant Income

## Checking for the Outliers.

In [ ]:
for i in Loan_train[num]:
    x=1
    axes=plt.subplots(x)
    sns.boxplot(Loan_train[i],ax=axes[x])
    x=x+1

In [ ]:
#We observe there are Significant Outliers

# Multivariate Analysis

In [ ]:
sns.pairplot(Loan_train[num],corner=True)

In [ ]:
sns.heatmap(Loan_train.corr(),annot=True,mask=np.triu(Loan_train.corr(),+1))

In [ ]:
# We observe there is no significan correlation Between the variables 

In [ ]:
cat_cols=['Gender','Married','Education','Self_Employed','Loan_Amount_Term','Credit_History','Property_Area']

In [ ]:
con_col=num

### Treating Null Values

In [ ]:
Loan_train['Gender']=Loan_train['Gender'].fillna(Loan_train['Gender'].mode()[0])
Loan_train['Married']=Loan_train['Married'].fillna(Loan_train['Married'].mode()[0])
Loan_train['Dependents']=Loan_train['Dependents'].fillna(Loan_train['Dependents'].mode()[0])
Loan_train['Self_Employed']=Loan_train['Self_Employed'].fillna(Loan_train['Self_Employed'].mode()[0])
Loan_train['LoanAmount']=Loan_train['LoanAmount'].fillna(Loan_train['LoanAmount'].median())
Loan_train['Loan_Amount_Term']=Loan_train['Loan_Amount_Term'].fillna(Loan_train['Loan_Amount_Term'].median())
Loan_train.loc[(Loan_train.Credit_History.isnull())&(Loan_train.ApplicantIncome>=3859),'Credit_History']=1
Loan_train.loc[(Loan_train.Credit_History.isnull())&(Loan_train.ApplicantIncome<=3859),'Credit_History']=0

# Encoding and Scaling

In [ ]:
Loan_train = pd.get_dummies(Loan_train, columns = cat_cols, drop_first = True)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
Loan_train[con_col] = scaler.fit_transform(Loan_train[con_col])
print("The first 5 rows of data are")
Loan_train.head()

# Train_Test_Split

In [ ]:
X=Loan_train.drop('Loan_Status',axis=1)
Y=Loan_train[['Loan_Status']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.30,stratify=Y)

In [ ]:
from sklearn import metrics

## Modeling selection 

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFCL=RandomForestClassifier(n_estimators=1000,max_depth=7,max_features='auto',min_samples_split=10,min_samples_leaf=5,random_state=1)
RFCL_model=RFCL.fit(X_train,Y_train)
print('Accuracy Score is ',RFCL_model.score(X_train,Y_train))
Y_train_predict_RFCL=RFCL_model.predict(X_train)
Y_test_predict_RFCL=RFCL_model.predict(X_test)
print('Accuracy Score is ',RFCL_model.score(X_test,Y_test))

In [ ]:
print("Confusion Matrix")
cm_test=metrics.confusion_matrix(Y_test, Y_test_predict_RFCL)
cm_test

In [ ]:
cr_test=metrics.classification_report(Y_test, Y_test_predict_RFCL)
print("Classification_report",'\n',cr_test )

In [ ]:
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression(solver='newton-cg',max_iter=10000)
LR_Model=LR.fit(X_train,Y_train)
print('Accuracy Score is ',LR_Model.score(X_train,Y_train))
Y_train_predict_LR=LR_Model.predict(X_train)
Y_test_predict_LR=LR_Model.predict(X_test)
print('Accuracy Score is ',LR_Model.score(X_test,Y_test))

In [ ]:
cm_lr_test=metrics.confusion_matrix(Y_test, Y_test_predict_LR)
print("Classification_report",'\n',cm_lr_test )

In [ ]:
cr_lr_test=metrics.classification_report(Y_test, Y_test_predict_LR)
print("Classification_report",'\n',cr_lr_test )

### Linear Discriminant Analysis 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
LDA=LinearDiscriminantAnalysis()
LDA_model=LDA.fit(X_train,Y_train)
LDA_train_model = LDA_model.score(X_train, Y_train)
Y_train_predict_LDA=LDA_model.predict(X_train)
print('Accuracy Score is ',LDA_train_model)
LDA_test_model = LDA_model.score(X_test, Y_test)
Y_test_predict_LDA=LDA_model.predict(X_test)
print('Accuracy Score is ',LDA_test_model)

In [ ]:
cm_lda_test=metrics.confusion_matrix(Y_test, Y_test_predict_LDA)
print("confusion_matrix",'\n',cm_lda_test )

In [ ]:
cr_lda_test=metrics.classification_report(Y_test, Y_test_predict_LDA)
print("Classification_report",'\n',cr_lda_test )

### K- Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier(n_neighbors=19,weights='distance' )
KNN_model=KNN.fit(X_train,Y_train)
KNN_train_model = KNN_model.score(X_train, Y_train)
print('Accuracy Score is ',KNN_train_model)
KNN_test_model = KNN_model.score(X_test, Y_test)
print('Accuracy Score is ',KNN_test_model)

In [ ]:
#We observe that KNN overfits the data

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB= GaussianNB()
NB_model = NB.fit(X_train,Y_train)
NB_train_model = NB_model.score(X_train, Y_train)
Y_train_predict_NB=NB_model.predict(X_train)
print('Accuracy Score is ',NB_train_model)
NB_test_model = NB_model.score(X_test, Y_test)
Y_test_predict_NB=NB_model.predict(X_test)
print('Accuracy Score is ',NB_test_model)


In [ ]:
cm_NB_test=metrics.confusion_matrix(Y_test, Y_test_predict_NB)
print("confusion_matrix",'\n',cm_NB_test )

In [ ]:
cr_NB_test=metrics.classification_report(Y_test, Y_test_predict_NB)
print("classification_report",'\n',cr_NB_test )

## Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
Bag = BaggingClassifier(n_estimators=20,random_state=1)
Bag_model = Bag.fit(X_train,Y_train)
Bag_train_model = Bag_model.score(X_train, Y_train)
print('Accuracy Score is ',Bag_train_model)
Bag_test_model = Bag_model.score(X_test, Y_test)
print('Accuracy Score is ',Bag_test_model)

In [ ]:
Y_train_predict_Bag=Bag_model.predict(X_train)
Y_test_predict_Bag=Bag_model.predict(X_test)

In [ ]:
cm_test_Bag=metrics.confusion_matrix(Y_test, Y_test_predict_Bag)
print("confusion_matrix",'\n',cm_test_Bag )

In [ ]:
cr_test_Bag=metrics.classification_report(Y_test, Y_test_predict_Bag)
print("classification_report",'\n',cr_test_Bag )

## AdaBoosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
Boost = AdaBoostClassifier(n_estimators=5, random_state=1)
Boost_model = Boost.fit(X_train,Y_train)
Boost_train_model = Boost_model.score(X_train, Y_train)
print('Accuracy Score is ',Boost_train_model)
Boost_test_model = Boost_model.score(X_test, Y_test)
print('Accuracy Score is ',Bag_test_model)

In [ ]:
Y_train_predict_Ad_boost=Boost_model.predict(X_train)
Y_test_predict_Ad_boost=Boost_model.predict(X_test)

In [ ]:
cm_test_Ad_boost=metrics.confusion_matrix(Y_test, Y_test_predict_Ad_boost)
print("confusion_matrix",'\n',cm_test_Ad_boost )

In [ ]:
cr_test_Ad_boost=metrics.classification_report(Y_test, Y_test_predict_Ad_boost)
print("classification_report",'\n',cr_test_Ad_boost )

### Gradient Boosting 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbcl=GradientBoostingClassifier(n_estimators=15,random_state=1)
gbcl_model = gbcl.fit(X_train,Y_train)
gbcl_train_model = gbcl_model.score(X_train, Y_train)
print('Accuracy Score is ',gbcl_train_model)
gbcl_test_model = gbcl_model.score(X_test, Y_test)
print('Accuracy Score is ',gbcl_test_model)

In [ ]:
Y_train_predict_Gradient_boost=Boost_model.predict(X_train)
Y_test_predict_Gradient_boost=Boost_model.predict(X_test)

In [ ]:
cm_test_Gradient_boost=metrics.confusion_matrix(Y_test, Y_test_predict_Gradient_boost)
print("confusion_matrix",'\n',cm_test_Gradient_boost )

In [ ]:
cr_test_Gradient_boost=metrics.classification_report(Y_test, Y_test_predict_Gradient_boost)
print("classification_report",'\n',cr_test_Gradient_boost )

### We observe that apart from KNN all the models give similar Accuracy but the recall and F1-Score is very low 
### Overall Looking at all the parameters Naive Bayes gives the best results